In [1]:
import pandas as pd

import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cityofnewyork.us", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.cityofnewyork.us,
#                  MyAppToken,
#                  userame="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("k397-673e", limit=2000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [5]:
results_df.describe()

,agency_name,agency_start_date,base_salary,first_name,fiscal_year,last_name,leave_status_as_of_july_31,mid_init,ot_hours,pay_basis,regular_gross_paid,regular_hours,title_description,total_ot_paid,total_other_pay,work_location_borough
count,2000,2000,2000,1872,2000,1872,2000,1400,2000,2000,2000,2000,2000,2000,2000,2000
unique,3,788,930,907,1,1568,4,25,329,6,1680,473,68,555,649,8
top,DISTRICT ATTORNEY-MANHATTAN,2016-06-06T00:00:00.000,1.00,MICHAEL,2016,SMITH,ACTIVE,A,0.00,per Annum,3500.00,1830.00,ASSISTANT DISTRICT ATTORNEY ...,0.00,0.00,MANHATTAN
freq,1773,83,142,41,2000,12,1618,181,1467,1799,47,784,668,1436,957,1773


In [6]:
from sklearn.model_selection import train_test_split

y = results_df.base_salary
df_features = ['fiscal_year', 'regular_hours','regular_gross_paid', 'ot_hours', 'agency_name', 'pay_basis','work_location_borough']
X = results_df[df_features]

train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=0)

In [20]:
s = (train_X.dtypes == 'object')
object_cols = list(s[s].index)
print s[s].index
print 'Category Variables:\n', object_cols

Index([u'fiscal_year', u'regular_hours', u'regular_gross_paid', u'ot_hours',
       u'agency_name', u'pay_basis', u'work_location_borough'],
      dtype='object')
Category Variables:
[u'fiscal_year', u'regular_hours', u'regular_gross_paid', u'ot_hours', u'agency_name', u'pay_basis', u'work_location_borough']


In [2]:
#Label Encoding --------------------------------------------------
#Label encoding assigns each unique value to a different integer.

#from sklearn.preprocessing import LabelEncoder

# Make copy to avoid changing original data 
#label_X_train = X_train.copy()
#label_X_valid = X_valid.copy()

# Apply label encoder to each column with categorical data
#label_encoder = LabelEncoder()
#for col in object_cols:
    #label_X_train[col] = label_encoder.fit_transform(X_train[col])
    #label_X_valid[col] = label_encoder.transform(X_valid[col])

In [38]:
#One-hot encoding creates new columns indicating the presence (or absence) of each possible value in the original
#data. To understand this, we'll work through an example.

from sklearn.preprocessing import OneHotEncoder

objects= ['agency_name', 'pay_basis', 'work_location_borough']

# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(train_X[objects]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(val_X[objects]))

# One-hot encoding removed index; put it back
OH_cols_train.index = train_X.index
OH_cols_valid.index = val_X.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = train_X.drop(objects, axis=1)
num_X_valid = val_X.drop(objects, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error

model = DecisionTreeRegressor(random_state=0)
model.fit(OH_X_train, train_y)
val_pred = model.predict(OH_X_valid)
mae = mean_absolute_error(val_y, val_pred)

#print val_pred.head()
print mae, '\n'
print val_pred[:5], '\n'
print val_y[:5]


4019.4502800000005 

[82500. 80000. 38180. 45641. 55666.] 

405     82000.00
1190    63047.00
1132    39841.00
731     45624.00
1754    55696.00
Name: base_salary, dtype: object


In [ ]:
#Missing Data:
#Three Approaches
#- 1) Drop Column: remove the column with missing data
#- 2) Imputation: add a value (the mean) to the missing spot
#- 3) Extention to Imputation: impute as before and add a column to show which values were imputed

#1)------------------------------------------------------------------------------------------
# Get names of columns with missing values
# cols_with_missing = [col for col in X_train.columns
                     #if X_train[col].isnull().any()]
    
# Drop columns in training and validation data
# reduced_X_train = X_train.drop(cols_with_missing, axis=1)
# reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)   

#2)------------------------------------------------------------------------------------------

#from sklearn.impute import SimpleImputer

# Imputation
# my_imputer = SimpleImputer()
# imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
# imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# Imputation removed column names; put them back
# imputed_X_train.columns = X_train.columns
# imputed_X_valid.columns = X_valid.columns

#3)------------------------------------------------------------------------------------------

# Make copy to avoid changing original data (when imputing)
# X_train_plus = X_train.copy()
# X_valid_plus = X_valid.copy()

# Make new columns indicating what will be imputed
# for col in cols_with_missing:
#    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
#    X_valid_plus[col + '_was_missing'] = X_valid_plus[col].isnull()

# Imputation
# my_imputer = SimpleImputer()
# imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
# imputed_X_valid_plus = pd.DataFrame(my_imputer.transform(X_valid_plus))

# Imputation removed column names; put them back
# imputed_X_train_plus.columns = X_train_plus.columns
# imputed_X_valid_plus.columns = X_valid_plus.columns



